In [1]:
!pip install cryptodataset==0.3.7 >> log.txt
!pip install stable-baselines3 >> log.txt

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gym
  DEPRECATION: gym was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https://github.com/pypa/pip/issues/8368
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cryptodataset 0.3.7 requires gym<0.27.0,>=0.26.2, but you have gym 0.21.0 which is incompatible.


In [2]:
from cryptodataset.envs import TradingEnv
from cryptodataset.evaluation import evaluate_model
from stable_baselines3 import PPO

In [6]:
env = TradingEnv(exchange='KuCoin', symbol='BTC/USDT')
model = PPO(policy='MlpPolicy', env=env)
for i in range(100):
    model.learn(5_000)
    # mean_reward, std_reward = evaluate_model(model, env, num_episodes=1)
    # print('#{}, mean_reward: {}, std_reward: {}'.format(i, mean_reward, std_reward))
    metrics = env.get_metrics()
    print(metrics)

2023-03-22 14:31:45.283 | INFO     | cryptodataset.ccxt:download_ohlcv:68 - data/KuCoin_BTCUSDT_1h.csv already exists, reading data from file


{'compsum': -0.6083818748482599, 'max_drawdown': -0.8580693043490674, 'sharpe': -0.043418042404877906}
{'compsum': 6.371708793289721, 'max_drawdown': -0.4373579329214038, 'sharpe': 0.4464200692031347}
{'compsum': 5.195783614764277, 'max_drawdown': -0.40896637869634256, 'sharpe': 0.41760359775555794}
{'compsum': -0.13395413117020172, 'max_drawdown': -0.7125023341936323, 'sharpe': 0.09419858754449355}
{'compsum': 0.2480160198916257, 'max_drawdown': -0.7406140656152467, 'sharpe': 0.1603413386804664}
{'compsum': 3.1139046316400334, 'max_drawdown': -0.6003112973401133, 'sharpe': 0.35329302138105084}
{'compsum': 5.1946347406942035, 'max_drawdown': -0.3764629929574057, 'sharpe': 0.42735570638812287}
{'compsum': 34.646289209364355, 'max_drawdown': -0.39801648789432575, 'sharpe': 0.727675620858793}
{'compsum': 15.038099482114426, 'max_drawdown': -0.37675247257603417, 'sharpe': 0.6178703983992796}
{'compsum': 0.9987852038783622, 'max_drawdown': -0.5607729905998499, 'sharpe': 0.23867657203582934}

In [7]:
env = TradingEnv(exchange='KuCoin', symbol='ETH/USDT')

2023-03-22 14:46:13.789 | INFO     | cryptodataset.ccxt:get_ohlcv:23 - fetching ETH/USDT ohlcv form KuCoin with timeframe 1h
2023-03-22 14:46:29.149 | INFO     | cryptodataset.ccxt:download_ohlcv:75 - saving ohlcv to data/KuCoin_ETHUSDT_1h.csv


In [8]:
def evaluate(model, env):
    obs = env.reset()

    done = False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
    print(env.get_metrics())

evaluate(model, env)

{'compsum': 657.8308633599319, 'max_drawdown': -0.6554063955867265, 'sharpe': 0.33921150858968924}


In [12]:
class BuyAndHold:
    def predict(self, obs, *args, **kwargs):
        return 2, None
evaluate(BuyAndHold(), env)

{'compsum': 4.803007559394317, 'max_drawdown': -0.9424938888111888, 'sharpe': 0.14730410514883358}
